# Cloning torchserve to add in the docker

In [1]:
# The dockerfile places the serve folder into the docker image
print("this will probably say serve already exists and that's okay just skip")
!git clone https://github.com/pytorch/serve.git

this will probably say serve already exists and that's okay just skip
fatal: destination path 'serve' already exists and is not an empty directory.


# Creating a docker image
### This creates a docker image from the dockerfile present in the current directory and pushes it to the ecr

In [2]:
import boto3, time, json
sess    = boto3.Session()
region  = sess.region_name
account = boto3.client('sts').get_caller_identity().get('Account')

In [3]:
import sagemaker

sagemaker_session = sagemaker.Session(boto_session=sess)

In [4]:
bucket_name = sagemaker_session.default_bucket()
print(bucket_name)
#bucket_name = "sagemaker-ap-southeast-1-614857483369"

sagemaker-us-west-1-096166425824


In [5]:
registry_name = 'torchserve-max'
!aws ecr create-repository --repository-name {registry_name}

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:us-west-1:096166425824:repository/torchserve-max",
        "registryId": "096166425824",
        "repositoryName": "torchserve-max",
        "repositoryUri": "096166425824.dkr.ecr.us-west-1.amazonaws.com/torchserve-max",
        "createdAt": 1625165235.0,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [6]:
image_label = 'v1'
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

!sudo docker build -t {registry_name}:{image_label} .
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}

Sending build context to Docker daemon  70.58MB
Step 1/25 : FROM ubuntu:18.04
 ---> 2eb2d388e1a2
Step 2/25 : ENV PYTHONUNBUFFERED TRUE
 ---> Using cache
 ---> 1e6af8e44e06
Step 3/25 : RUN apt-get update &&     DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y     fakeroot     ca-certificates     dpkg-dev     g++     python3-dev     openjdk-11-jdk     curl     vim     git     && rm -rf /var/lib/apt/lists/*     && cd /tmp     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> ed5f3771c9fd
Step 4/25 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
 ---> Using cache
 ---> abd3a28fb31d
Step 5/25 : RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
 ---> Using cache
 ---> 8dc854a52793
Step 6/25 : RUN pip install aiohttp
 ---> Using cache
 ---> 54064956ae1d
Step 7/25 : RUN pip install zmq
 ---> Using cache
 ---> 09f1453ee5f6
Step 8/25 : RUN pip install fairseq
 ---> 

Removing intermediate container 30bc87b2e2f4
 ---> 676a4ae5695f
Step 15/25 : COPY dockerd-entrypoint.sh /usr/local/bin/dockerd-entrypoint.sh
 ---> fc49cf22c382
Step 16/25 : RUN chmod +x /usr/local/bin/dockerd-entrypoint.sh
 ---> Running in cab756e97bfa
Removing intermediate container cab756e97bfa
 ---> f0245757a356
Step 17/25 : RUN mkdir -p /home/model-server/ && mkdir -p /home/model-server/tmp
 ---> Running in 3441053de1e4
Removing intermediate container 3441053de1e4
 ---> 9411dec788ae
Step 18/25 : COPY config.properties /home/model-server/config.properties
 ---> 4aa6bfa630ea
Step 19/25 : WORKDIR /home/model-server
 ---> Running in cc0868601e8c
Removing intermediate container cc0868601e8c
 ---> 4e2653e4d9ea
Step 20/25 : ENV TEMP=/home/model-server/tmp
 ---> Running in f5b44dccc854
Removing intermediate container f5b44dccc854
 ---> f3ca3572fd34
Step 21/25 : ADD anli-private /home/model-server/anli-old
 ---> a1f46a03aed2
Step 22/25 : ADD anli-public /home/model-server/anli
 ---> 997cbb0

d0b3ebb: Pushing  1.726GB/2.717GBPushing  538.1MB/2.717GB

d0b3ebb: Pushed   2.723GB/2.717GBPushing   2.16GB/2.717GBPushing  2.312GB/2.717GBv1: digest: sha256:b3db9d6aea380e9af9f78ac5a48db22f19a2cc30f86b7330047084848f72ce9b size: 4936
